# Installing Required Libraries

In [ ]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

# Importing Libraries and Warning Control

In [ ]:
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

# Warning control
import warnings
warnings.filterwarnings('ignore')

# Defining the LLM (Language Model) & Serper Tool:

In [ ]:
llm = ChatGroq(temperature=0,
             model_name="llama3-70b-8192",
             api_key='You should write your API KEY here.')
import os
# Serper API anahtarını doğrudan çevresel değişken olarak ayarlayın
os.environ["SERPER_API_KEY"] = "You should write your SERPER API KEY here."

# Creating Agents

In [ ]:
sales_rep_agent = Agent(
		llm = llm,
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
		llm = llm,
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

# Creating Tools

**crewAI Tools**

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

**Custom Tool**

*   Create a custom tool using crewAi's BaseTool class

In [ ]:
from crewai_tools import BaseTool


*   Every Tool needs to have a name and a description.
*   For simplicity and classroom purposes, SentimentAnalysisTool will return positive for every text.
*   When running locally, you can customize the code with your logic in the _run function.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

# Creating Tasks

**The Lead Profiling Task is using crewAI Tools.**

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

**The Personalized Outreach Task is using your custom Tool SentimentAnalysisTool, as well as crewAI's SerperDevTool (search_tool).**

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

# Creating the Crew

**Memory**

*   Setting memory=True when putting the crew together enables Memory.

**"memory=True" Error Explanation**
* The memory=True parameter enables memory features and maintains context between tasks. Removing this parameter causes tasks to run independently and eliminates additional memory management operations.

* If memory management is not configured correctly or if the Groq API key does not support certain memory management features, errors may occur.

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	# memory=True
)

# Running the Crew

**Guardrails**

*   By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by exploring the available files in the directory to see if there's any information about DeepLearningAI.

Action: List files in directory
Action Input: {} 

File paths: 
-

Thought: It seems like there are no files directly related to DeepLearningAI in the directory. I need to think of a different approach.

Action: Read a file's content
Action Input: {"file_path": "./instruction

# Display the results of your execution as markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

**Email Draft 1: Congratulations on the Recent Product Launch**

Subject: Congratulations on the Recent Product Launch, Andrew!

Dear Andrew,

I wanted to reach out and congratulate you on the recent product launch at DeepLearningAI. It's impressive to see how your company is revolutionizing the education technology space by developing a global community of AI talent.

As a pioneer in machine learning and online education, I'm sure you're committed to providing the best possible learning experiences for your students. Our solutions at CrewAI can help you achieve this goal by improving student engagement and retention, and providing personalized learning experiences.

I'd love to discuss how our solutions can support your goals and help you scale your online courses. Would you be open to a demo or a call to explore this further?

Best regards,
[Your Name]

**Email Draft 2: Personalized Learning Experiences**

Subject: Enhancing Student Engagement with Personalized Learning Experiences

Hi Andrew,

As someone who has authored or co-authored over 200 research papers in AI, I'm sure you understand the importance of providing personalized learning experiences for your students. Our solutions at CrewAI can help you achieve this by leveraging AI-powered tools to create customized learning paths for each student.

I'd love to share some case studies of similar companies that have benefited from our solutions and explore how we can help DeepLearningAI achieve its goals.

Would you be open to a call to discuss this further?

Best regards,
[Your Name]

**Email Draft 3: Scaling Online Courses**

Subject: Scaling Your Online Courses with AI-Powered Solutions

Dear Andrew,

As the founder of DeepLearningAI, you must be committed to reaching a wider audience and scaling your online courses. Our solutions at CrewAI can help you achieve this by providing AI-powered tools to improve student engagement and retention.

I'd love to offer a free trial or pilot project to test our solutions and demonstrate their value. Would you be open to exploring this further?

Best regards,
[Your Name]

These email drafts are personalized, engaging, and aligned with DeepLearningAI's corporate identity. They address Andrew Ng's interests and goals, and demonstrate a deep understanding of the company's needs and values.